In [1]:
import polars as pl
import pandas as pd

from pandas_datareader import data as pdr
import yfinance as yf
yf.pdr_override()

# Get data from yahoo

In [2]:
ticker = 'AMD'
df = yf.download(ticker, interval = '1d', start='2010-01-01')

[*********************100%***********************]  1 of 1 completed


# Save to disk

In [3]:
df.to_csv(f'data/daily/{ticker}.csv')

In [4]:
len(df)

3420

## The csv file is 375k for the dates up to August 4, 2023

In [5]:
!dir .\\data\\daily

 Volume in drive F is TS1
 Volume Serial Number is B4DA-75AE

 Directory of F:\chris\workspace\Notebooks\data\daily

08/06/2023  08:14 PM    <DIR>          .
08/06/2023  08:14 PM    <DIR>          ..
08/06/2023  08:14 PM           374,027 AMD.csv
               1 File(s)        374,027 bytes
               2 Dir(s)  282,010,320,896 bytes free


## Instead, save as parquet

In [6]:
df.to_parquet(f'data/daily/{ticker}.parquet')

In [7]:
!dir .\\data\\daily

 Volume in drive F is TS1
 Volume Serial Number is B4DA-75AE

 Directory of F:\chris\workspace\Notebooks\data\daily

08/06/2023  08:14 PM    <DIR>          .
08/06/2023  08:14 PM    <DIR>          ..
08/06/2023  08:14 PM           374,027 AMD.csv
08/06/2023  08:14 PM           147,147 AMD.parquet
               2 File(s)        521,174 bytes
               2 Dir(s)  282,010,173,440 bytes free


# Let's switch to polars and clean up the columns

In some instances, yf can return a fload for the volume. This can be a problem if you intend to search multiple files (as we will) and in one file, the volume is a float while another file is an int.  Also, searching multiple files is easier if we add the symbol as a column.

In [8]:
df_pl = (pl.from_pandas(df.reset_index())
    # add a column for the symbol
    # cast the volume column to a type int64
    .with_columns([
        pl.lit(ticker).alias("Symbol"),
        pl.col("Volume").cast(pl.Int64)
    ])
)

In [9]:
df_pl.head(5)

Date,Open,High,Low,Close,Adj Close,Volume,Symbol
datetime[ns],f64,f64,f64,f64,f64,i64,str
2010-01-04 00:00:00,9.79,9.9,9.68,9.7,9.7,18748700,"""AMD"""
2010-01-05 00:00:00,9.71,9.9,9.68,9.71,9.71,22145700,"""AMD"""
2010-01-06 00:00:00,9.68,9.76,9.55,9.57,9.57,18643400,"""AMD"""
2010-01-07 00:00:00,9.51,9.55,9.18,9.47,9.47,26806800,"""AMD"""
2010-01-08 00:00:00,9.37,9.47,9.29,9.43,9.43,13752800,"""AMD"""


In [10]:
df_pl.write_parquet(f'data/daily/{ticker}.parquet')

In [11]:
!dir .\\data\\daily

 Volume in drive F is TS1
 Volume Serial Number is B4DA-75AE

 Directory of F:\chris\workspace\Notebooks\data\daily

08/06/2023  08:14 PM    <DIR>          .
08/06/2023  08:14 PM    <DIR>          ..
08/06/2023  08:14 PM           374,027 AMD.csv
08/06/2023  08:14 PM            92,717 AMD.parquet
               2 File(s)        466,744 bytes
               2 Dir(s)  282,010,226,688 bytes free


## Let's do it again but in 1 line

In [12]:
ticker = 'TLT'
(pl.from_pandas(yf.download(ticker, interval = '1d', start='2010-01-01', progress=False).reset_index())
    # add a column for the symbol
    # cast the volume column to a type int64
    .with_columns([
        pl.lit(ticker).alias("Symbol"),
        pl.col("Volume").cast(pl.Int64)
    ])
    .write_parquet(f'data/daily/{ticker}.parquet')
)

In [13]:
!dir .\\data\\daily

 Volume in drive F is TS1
 Volume Serial Number is B4DA-75AE

 Directory of F:\chris\workspace\Notebooks\data\daily

08/06/2023  08:14 PM    <DIR>          .
08/06/2023  08:14 PM    <DIR>          ..
08/06/2023  08:14 PM           374,027 AMD.csv
08/06/2023  08:14 PM            92,717 AMD.parquet
08/06/2023  08:14 PM            93,070 TLT.parquet
               3 File(s)        559,814 bytes
               2 Dir(s)  282,010,132,480 bytes free


In [14]:
(pl.scan_parquet('data/daily/*.parquet')
    .groupby(['Symbol'])
    .agg(
        pl.col("Date").min().alias("First Date"),
        pl.col("Date").max().alias("Last Date")
    )
    .collect()
)

Symbol,First Date,Last Date
str,datetime[ns],datetime[ns]
"""AMD""",2010-01-04 00:00:00,2023-08-04 00:00:00
"""TLT""",2010-01-04 00:00:00,2023-08-04 00:00:00


In [15]:
ticker = 'SNOW'
(pl.from_pandas(yf.download(ticker, interval = '1d', start='2010-01-01', progress=False).reset_index())
    # add a column for the symbol
    # cast the volume column to a type int64
    .with_columns([
        pl.lit(ticker).alias("Symbol"),
        pl.col("Volume").cast(pl.Int64)
    ])
    .write_parquet(f'data/daily/{ticker}.parquet')
)

In [16]:
(pl.scan_parquet('data/daily/*.parquet')
    .groupby(['Symbol'])
    .agg(
        pl.col("Date").min().alias("First Date"),
        pl.col("Date").max().alias("Last Date")
    )
    .collect()
)

Symbol,First Date,Last Date
str,datetime[ns],datetime[ns]
"""SNOW""",2020-09-16 00:00:00,2023-08-04 00:00:00
"""AMD""",2010-01-04 00:00:00,2023-08-04 00:00:00
"""TLT""",2010-01-04 00:00:00,2023-08-04 00:00:00
